# Tutorials python code

In [1]:
#esrgan
!git clone https://github.com/xinntao/Real-ESRGAN.git

Cloning into 'Real-ESRGAN'...


In [2]:
%cd C:\CCC\Real-ESRGAN

C:\CCC\Real-ESRGAN


In [ ]:
# liberary install
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt

In [4]:
!python setup.py develop

running develop
running egg_info
creating realesrgan.egg-info
writing realesrgan.egg-info\PKG-INFO
writing dependency_links to realesrgan.egg-info\dependency_links.txt
writing requirements to realesrgan.egg-info\requires.txt
writing top-level names to realesrgan.egg-info\top_level.txt
writing manifest file 'realesrgan.egg-info\SOURCES.txt'
reading manifest file 'realesrgan.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'realesrgan.egg-info\SOURCES.txt'
running build_ext
Creating c:\users\user\.conda\envs\kdb\lib\site-packages\realesrgan.egg-link (link to .)
Adding realesrgan 0.3.0 to easy-install.pth file

Installed c:\ccc\real-esrgan
Processing dependencies for realesrgan==0.3.0
Searching for tqdm==4.65.0
Best match: tqdm 4.65.0
Adding tqdm 4.65.0 to easy-install.pth file
Installing tqdm-script.py script to c:\Users\User\.conda\envs\kdb\Scripts
Installing tqdm.exe script to c:\Users\User\.conda\envs\kdb\Scripts

Using 

c:\Users\User\.conda\envs\kdb\lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
c:\Users\User\.conda\envs\kdb\lib\site-packages\setuptools\command\develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
 

In [ ]:
''' Usage : python inference_realesrgan.py -n RealESRGAN_x4plus -i infile -o outfile [options]...

A common command: python inference_realesrgan.py -n RealESRGAN_x4plus -i infile --outscale 3.5 --face_enhance

  -h                   show this help
  -i --input           Input image or folder. Default: inputs
  -o --output          Output folder. Default: results
  -n --model_name      Model name. Default: RealESRGAN_x4plus
  -s, --outscale       The final upsampling scale of the image. Default: 4
  --suffix             Suffix of the restored image. Default: out
  -t, --tile           Tile size, 0 for no tile during testing. Default: 0
  --face_enhance       Whether to use GFPGAN to enhance face. Default: False
  --fp32               Use fp32 precision during inference. Default: fp16 (half precision).
  --ext                Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto
'''

input : Aduino_pico_camera pictures data  
output : Aduino_pico_camera pictures data.upscaling data


In [6]:
# inference
!python inference_realesrgan.py --model_name RealESRGAN_x4plus --input 'input source' --output 'output source' --outscale 4
#--face_enhance False 기본값

c:\Users\User\.conda\envs\kdb\lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


# image Captioning

In [ ]:
import os
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [ ]:
#hugging face model load
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [ ]:
#cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [ ]:
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
#inference
image_dir = 'C:\\esrganapi\\Real-ESRGAN\\output source'
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
captions = predict_step(image_files)
print(captions)

In [ ]:
import shutil
import os 
# gpt가 짜주는 사진이름 저장 
image_dir = 'C:\\esrganapi\\Real-ESRGAN\\output source'
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]

for image_file in image_files:
    caption = predict_step([image_file])
    # Convert the predicted caption to a form that can be used as a file name
    # Remove special characters, replace spaces with '_', etc.
    safe_caption = caption.replace(" ", "_").replace("/", "_").replace("\\", "_").replace(":", "_").replace("*", "_").replace("?", "_").replace("\"", "_").replace("<", "_").replace(">", "_").replace("|", "_")
    new_file_name = safe_caption + '.jpg'
    new_file_path = os.path.join(image_dir, new_file_name)
    shutil.move(image_file, new_file_path)